In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers


Matplotlib is building the font cache; this may take a moment.


In [2]:
# Load dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Train/Validation/Test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


In [3]:
def build_model(input_dim, lr, num_layers, neurons, dropout, weight_decay):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))

    for _ in range(num_layers):
        model.add(layers.Dense(
            neurons,
            activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
        ))
        model.add(layers.Dropout(dropout))

    model.add(layers.Dense(1))

    optimizer = optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae']
    )
    return model


In [ ]:
import random

def sample_config():
    return {
        "lr": 10 ** random.uniform(-5, -2),
        "batch_size": ([16]),
        "num_layers": random.randint(6, 10),
        "neurons": random.choice([32, 64, 128, 256]),
        "dropout": random.uniform(0.0, 0.5),
        "weight_decay": 10 ** random.uniform(-6, -2)
    }


In [5]:
configs = [sample_config() for _ in range(64)]
history_store = {}
results = []

epochs_schedule = [5, 15, 30]

for stage, epochs in enumerate(epochs_schedule):
    print(f"\nStage {stage+1} - Training for {epochs} epochs")

    stage_results = []

    for i, cfg in enumerate(configs):
        model = build_model(
            X_train.shape[1],
            cfg["lr"],
            cfg["num_layers"],
            cfg["neurons"],
            cfg["dropout"],
            cfg["weight_decay"]
        )

        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=cfg["batch_size"],
            verbose=0
        )

        val_loss = history.history["val_loss"][-1]
        stage_results.append((cfg, val_loss, history))
        history_store[str(cfg)] = history

    stage_results.sort(key=lambda x: x[1])
    configs = [cfg for cfg, _, _ in stage_results[:len(stage_results)//2]]



Stage 1 - Training for 5 epochs

Stage 2 - Training for 15 epochs

Stage 3 - Training for 30 epochs


KeyboardInterrupt: 

In [ ]:
best_8 = stage_results[:8]

plt.figure(figsize=(10,6))
for i, (cfg, _, hist) in enumerate(best_8):
    plt.plot(hist.history['loss'], label=f'Config {i+1}')

plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.title("Loss vs Epochs for Best 8 Configurations")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def evaluate_model(cfg):
    model = build_model(
        X_train.shape[1],
        cfg["lr"],
        cfg["num_layers"],
        cfg["neurons"],
        cfg["dropout"],
        cfg["weight_decay"]
    )
    model.fit(X_train, y_train, epochs=30, batch_size=cfg["batch_size"], verbose=0)

    preds = model.predict(X_test).flatten()
    return {
        "R2": r2_score(y_test, preds),
        "MAE": mean_absolute_error(y_test, preds),
        "MSE": mean_squared_error(y_test, preds)
    }

metrics_table = []
for i, (cfg, _, _) in enumerate(best_8):
    metrics = evaluate_model(cfg)
    metrics_table.append([i+1, "MSE", metrics["R2"], metrics["MAE"], metrics["MSE"]])

metrics_df = pd.DataFrame(
    metrics_table,
    columns=["Configuration #", "Loss Function", "R² Score", "MAE", "MSE"]
)
metrics_df


In [ ]:
best_4 = best_8[:4]

hp_table = []
for i, (cfg, _, _) in enumerate(best_4):
    hp_table.append([
        i+1,
        cfg["lr"],
        cfg["batch_size"],
        cfg["num_layers"],
        cfg["neurons"],
        cfg["dropout"],
        cfg["weight_decay"]
    ])

hp_df = pd.DataFrame(
    hp_table,
    columns=[
        "Configuration #",
        "Learning rate",
        "Batch size",
        "Hidden layers",
        "Neurons per layer",
        "Dropout rate",
        "Weight decay"
    ]
)
hp_df


Hyperparameter tuning significantly improves model performance. Random search combined with successive halving efficiently explored the search space while reducing computational cost.

Lower learning rates, deeper networks, and moderate regularization consistently produced better generalization performance.